In [1]:
from __future__ import print_function, division
import numpy as np
import os
import gc
import sys
import datetime
import h5py
import contextlib
import math
import pandas as pd
from IPython.display import display

C:\Users\Landscape\Anaconda3\envs\PythonDLCPU\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from model_specification import *
from data_interface import *
import learning_parameters

Using TensorFlow backend.


In [32]:
# Settings
evaluation_file_path = "../data/dataset/Eval_20180204_2030.h5"
dataset_partition = "testing"
interpretation_file_path = "../data/dataset/interpretation_20180204.h5"

first_stage_verbose = True

In [4]:
evaluation_data_interface = EvaluationDataInterface()
evaluation_data_interface.open_file(evaluation_file_path)
interpretation_data_interface = EvaluationDataInterface()
interpretation_data_interface.open_file(interpretation_file_path)

In [6]:
table_text, count_cross_table, prob_label_cross_table, prob_pred_cross_table = evaluation_data_interface.read_evaluation_results(dataset_partition)

label_descriptions = evaluation_data_interface.read_label_descriptions()
label_description_list = [''] * count_cross_table.shape[1]
for label_id, label_description in label_descriptions:
    label_description_list[int(label_id)] = label_description

sum_table_label = count_cross_table.sum(axis=-2)
sum_table_pred = count_cross_table.sum(axis=-1)

interpretation_headers, interpretation_scores = interpretation_data_interface.read_interpretation_results()


In [7]:
evaluation_data_interface.close_file()
interpretation_data_interface.close_file()

In [8]:
for i, data_name in enumerate(table_text):
    print(i, ":", data_name)

0 : overall
1 : 8
2 : 20
3 : 49
4 : 67
5 : 77
6 : 85
7 : 91
8 : 117
9 : 119


In [22]:
read_index = range(len(table_text))
# read_index = [1]

In [23]:
bacteria_type_mapping = [[1], [2], [3]]
interpretation_str_list = ["Normal", "Intermediate", "Infection"]

def get_bacteria_count_sums(bacteria_count_full, bacteria_type_mapping):
    results = []
    for list_labels in bacteria_type_mapping:
        count_total = 0
        for label in list_labels:
            count_total += bacteria_count_full[label]
        results.append(count_total)
    return results

def get_density_class(count):
    if count == 0:
        return 0
    elif count < 1:
        return 1
    elif count < 5:
        return 2
    elif count <= 30:
        return 3
    else:
        return 4

def get_nugent_score(lactobacillus_count, gardnerella_bacteroides_count, curved_rod_count):
    total_score = 0
    lactobacillus_score = 4 - get_density_class(lactobacillus_count)
    gardnerella_score = get_density_class(gardnerella_bacteroides_count)
    curved_rod_score = int( (get_density_class(curved_rod_count) + 1) / 2)
    return lactobacillus_score, gardnerella_score, curved_rod_score, lactobacillus_score + gardnerella_score + curved_rod_score
    
def get_nugent_score_interpretation_int(nugent_score):
    if nugent_score <= 3:
        return 0
    elif nugent_score <= 6:
        return 1
    else:
        return 2

def get_nugent_score_interpretation_str(nugent_score):
    if nugent_score <= 3:
        return "Normal"
    elif nugent_score <= 6:
        return "Intermediate"
    else:
        return "Infection"
    
@contextlib.contextmanager
def print_options_context(*args, **kwargs):
    original_print_options = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally: 
        np.set_printoptions(**original_print_options)
        

def dataframe_highlight_diagonal(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    num_rows = len(data)
    is_diagonal = [[x == y for x in range(num_rows)] for y in range(num_rows)]
    
    return pd.DataFrame(np.where(is_diagonal, attr, ''), index=data.index, columns=data.columns)
        
def dataframe_suppress_zero(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    attr = 'color: {}'.format('lightgray' if math.isclose(0.0, val, rel_tol=1e-5) else 'black')
    return attr
    
def get_pandas_dataframe(cross_table, axis_text_list = None, highlight_diagonal = False, fixed_width = None):
    df = pd.DataFrame(cross_table, index = axis_text_list, columns = axis_text_list).style.\
            applymap(dataframe_suppress_zero)
        
    if highlight_diagonal:
        df = df.apply(dataframe_highlight_diagonal, color='yellow', axis=None)

#     if not fixed_width is None:
#         df = df.applymap(lambda x: str(x).ljust(fixed_width))
#         df.columns = df.columns.map(lambda x: str(x).ljust(fixed_width))
        
    return df


In [34]:
lactobacillus_score_corss_table = np.zeros((5, 5), dtype=int)
gardnerella_score_cross_table = np.zeros((5, 5), dtype=int)
curved_rod_score_cross_table = np.zeros((3, 3), dtype=int)
nugent_score_cross_table = np.zeros((11, 11), dtype=int)
infection_degree_cross_table = np.zeros((3, 3), dtype=int)

with print_options_context(formatter={'float': '{:>7.1%}'.format}):
    for i in read_index:
        
        print("Result for " + table_text[i])
        if first_stage_verbose:
            print("Count cross table")
            display(get_pandas_dataframe(count_cross_table[i], label_description_list, highlight_diagonal = True, fixed_width = 11))
            print("Probability of given label")
            display(get_pandas_dataframe(prob_label_cross_table[i], label_description_list, highlight_diagonal = True).format("{:>7.1%}"))
            print("Probability of given prediction")
            display(get_pandas_dataframe(prob_pred_cross_table[i], label_description_list, highlight_diagonal = True).format("{:>7.1%}"))

        print("Prediction Nugent Score and degree of infection")
        bacteria_counts = get_bacteria_count_sums(sum_table_pred[i], bacteria_type_mapping)
        pred_lscore, pred_gscore, pred_cscore, pred_total = get_nugent_score(bacteria_counts[0], bacteria_counts[1], bacteria_counts[2])
        score_interpretation_str = get_nugent_score_interpretation_str(pred_total)
        print("Score: {:d} + {:d} + {:d} = {:2d}, Interpretation: {:s}".format(pred_lscore, pred_gscore, pred_cscore, pred_total, score_interpretation_str))

        print("Label Nugent Score and degree of infection")
        bacteria_counts = get_bacteria_count_sums(sum_table_label[i], bacteria_type_mapping)
        label_lscore, label_gscore, label_cscore, label_total = get_nugent_score(bacteria_counts[0], bacteria_counts[1], bacteria_counts[2])
        score_interpretation_str = get_nugent_score_interpretation_str(label_total)
        print("Score: {:d} + {:d} + {:d} = {:2d}, Interpretation: {:s}".format(label_lscore, label_gscore, label_cscore, label_total, score_interpretation_str))

        if table_text[i] in interpretation_scores:
            print("Source Nugent Score and degree of infection")
            source_lscore, source_gscore, source_cscore = interpretation_scores[table_text[i]]
            source_total = source_lscore + source_gscore + source_cscore
            score_interpretation_str = get_nugent_score_interpretation_str(source_total)
            print("Score: {:d} + {:d} + {:d} = {:2d}, Interpretation: {:s}".format(source_lscore, source_gscore, source_cscore, source_total, score_interpretation_str))
            
            lactobacillus_score_corss_table[source_lscore, pred_lscore] += 1
            gardnerella_score_cross_table[source_gscore, pred_gscore] += 1
            curved_rod_score_cross_table[source_cscore, pred_cscore] += 1
            nugent_score_cross_table[source_total, pred_total] += 1
            infection_degree_cross_table[get_nugent_score_interpretation_int(source_total), get_nugent_score_interpretation_int(pred_total)] += 1
        else:
            print("No source data available for " + table_text[i])
            
        print("----------------------------------------")

Result for overall
Count cross table


,,Lactobacillus,Gardnerella,Curved Rod,Unknown A,Coccus,Yeast,Noise - Edge,Noise - Decompose
,0,581,275,0,0,0,0,8,17
Lactobacillus,0,36,2,0,0,0,0,1,0
Gardnerella,0,86,31,0,0,0,0,0,0
Curved Rod,0,4,0,0,0,0,0,0,0
Unknown A,0,0,0,0,0,0,0,0,0
Coccus,0,0,0,0,0,0,0,0,0
Yeast,0,0,0,0,0,0,0,0,0
Noise - Edge,0,7,18,0,0,0,0,15,7
Noise - Decompose,0,8,12,0,0,0,0,0,4


Probability of given label


,,Lactobacillus,Gardnerella,Curved Rod,Unknown A,Coccus,Yeast,Noise - Edge,Noise - Decompose
,0.0%,65.9%,31.2%,0.0%,0.0%,0.0%,0.0%,0.9%,1.9%
Lactobacillus,0.0%,92.3%,5.1%,0.0%,0.0%,0.0%,0.0%,2.6%,0.0%
Gardnerella,0.0%,73.5%,26.5%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Curved Rod,0.0%,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Unknown A,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Coccus,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Yeast,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Noise - Edge,0.0%,14.9%,38.3%,0.0%,0.0%,0.0%,0.0%,31.9%,14.9%
Noise - Decompose,0.0%,33.3%,50.0%,0.0%,0.0%,0.0%,0.0%,0.0%,16.7%


Probability of given prediction


...

Probability of given label


,,Lactobacillus,Gardnerella,Curved Rod,Unknown A,Coccus,Yeast,Noise - Edge,Noise - Decompose
,0.0%,74.5%,25.5%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Lactobacillus,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Gardnerella,0.0%,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Curved Rod,0.0%,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Unknown A,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Coccus,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Yeast,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Noise - Edge,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Noise - Decompose,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%


Probability of given prediction


,,Lactobacillus,Gardnerella,Curved Rod,Unknown A,Coccus,Yeast,Noise - Edge,Noise - Decompose
,0.0%,97.4%,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Lactobacillus,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Gardnerella,0.0%,1.9%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Curved Rod,0.0%,0.6%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Unknown A,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Coccus,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Yeast,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Noise - Edge,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Noise - Decompose,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%


Prediction Nugent Score and degree of infection
Score: 4 + 2 + 1 =  7, Interpretation: Infection
Label Nugent Score and degree of infection
Score: 0 + 4 + 0 =  4, Interpretation: Intermediate
Source Nugent Score and degree of infection
Score: 2 + 4 + 1 =  7, Interpretation: Infection
----------------------------------------


In [35]:
print("Lactobacillus score cross table")
display(get_pandas_dataframe(lactobacillus_score_corss_table, highlight_diagonal = True))
print("Gardnerella score cross table")
display(get_pandas_dataframe(gardnerella_score_cross_table, highlight_diagonal = True))
print("Curved rod score cross table")
display(get_pandas_dataframe(curved_rod_score_cross_table, highlight_diagonal = True))
print("Total nugent score cross table")
display(get_pandas_dataframe(nugent_score_cross_table, highlight_diagonal = True))
print("Infection degree cross table")
display(get_pandas_dataframe(infection_degree_cross_table, interpretation_str_list, highlight_diagonal = True))

Lactobacillus score cross table


,0,1,2,3,4
0,0,0,0,0,0
1,0,0,0,0,2
2,0,0,0,0,1
3,0,0,0,0,3
4,0,0,0,0,3


Gardnerella score cross table


,0,1,2,3,4
0,1,0,0,0,0
1,0,0,0,0,0
2,0,0,1,0,0
3,0,0,3,0,0
4,0,0,3,1,0


Curved rod score cross table


,0,1,2
0,0,4,0
1,0,4,0
2,0,1,0


Total nugent score cross table


,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,2,0,0,0
6,0,0,0,0,0,0,0,2,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,2,0,0,0


Infection degree cross table


,Normal,Intermediate,Infection
Normal,0,1,0
Intermediate,0,0,4
Infection,0,0,4
